In [1]:
import pandas as pd
import requests
import json
import altair as alt
import numpy as np

In [2]:
api_key = "RGAPI-ea6f4e66-adb6-4295-9551-99053de9dee5"

In [6]:
puuid = "C5pz-r_LXeHapwqxflnPxizTjQ2HDYrwkRaa7plZshcRjfEk0SZv3--vFs1QBqevX8Zmw1tVffwZbg"

In [7]:
matches = requests.get(f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20&api_key={api_key}").json()

In [8]:
match = matches[3]

['NA1_4940766256',
 'NA1_4940540609',
 'NA1_4940531153',
 'NA1_4940509479',
 'NA1_4940257601',
 'NA1_4940247188',
 'NA1_4939875515',
 'NA1_4939852486',
 'NA1_4939067609',
 'NA1_4939037635',
 'NA1_4938283583',
 'NA1_4938262168',
 'NA1_4937556276',
 'NA1_4937516691',
 'NA1_4937308099',
 'NA1_4937272459',
 'NA1_4936362993',
 'NA1_4936334720',
 'NA1_4936324083',
 'NA1_4935936622']

In [15]:
gold_time = pd.DataFrame(columns = ['Champion', 'Time', "Gold"])

In [11]:
def get_match_gold(matchId, puuid): #Given a matchId and a player's puuid, get their gold income over time for that game.
    time_re = requests.get(f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}/timeline?api_key={api_key}")
    match_re = requests.get(f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}")

    timeline = time_re.json()['info']
    match = match_re.json()['info']


    participant = pd.DataFrame(timeline['participants']).set_index('puuid').loc[puuid].get('participantId') #Get the participantId of the player with given puuid

    champ_names = {player['puuid'] : player['championName'] for player in match['participants']} #Get the names of everyone's champions

    times = [(champ_names[puuid], np.round(frame['timestamp'] / 60000, 0) , frame['participantFrames'][f'{participant}']['totalGold'])  #Get the timestamps and gold at each timestamp for given participantId
             for frame in timeline['frames']]
    
    return pd.DataFrame(times, columns = ['Champion', 'Time', 'Gold']) #Transform to DataFrame and return


In [16]:
for match in matches[:10]:
    gold_time = pd.concat([gold_time, get_match_gold(match, puuid)])

In [17]:
gold_time

,Champion,Time,Gold
0,Aatrox,0.0,500
1,Aatrox,1.0,500
2,Aatrox,2.0,584
3,Aatrox,3.0,885
4,Aatrox,4.0,1099
...,...,...,...
20,Aatrox,20.0,9522
21,Aatrox,21.0,10626
22,Aatrox,22.0,10922
23,Aatrox,23.0,11468


In [18]:
gold_time_final = gold_time.groupby(['Champion', 'Time'])['Gold'].mean().reset_index()

In [19]:
alt.Chart(gold_time_final).mark_line().encode(
    x = alt.X('Time'),
    y = alt.Y('Gold'),
    color = alt.Color('Champion')
)

alt.Chart(...)